In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd
from textblob import TextBlob
from nltk import word_tokenize

mylist = list()
InputSheet = "Same-Sim_SAMPLE.csv"
MasterSheet = "Same_SIM_Master_List.csv"
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy','Rank','Comments'),index=range(30000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv(MasterSheet)
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def Get_SimCodeDesc():
    df = pd.read_csv(InputSheet,encoding ='latin1')
    sMasterCodeDesc = df['Sim_LongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetSameSIMData():
    SameSimDf = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = SameSimDf['SimCode'].tolist()
    return sNewCodeDesc;
   
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

def GetRowNum(): 
    try:
        rownum = resultFrame.loc[(resultFrame.Accuracy == '0003T') & (resultFrame.Rank == 'CERVICOGRAPY')].index[0]
            #print(rownum)
        resultFrame.loc[rownum,'Comments'] = "Test"
    except:
        "No record found"

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,RankNo,Comment,x):
    df = pd.read_csv(MasterSheet)    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    resultFrame.loc[x].Rank = RankNo
    resultFrame.loc[x].Comments = Comment
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
SimCodeDescSet = pd.Series(Get_SimCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
SameSimSet = pd.Series(GetSameSIMData())
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    test_set = pd.Series(i)   
    try:
        rowFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum])].index[0]  
        try:        
            rowNumSameSimFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum]) & (resultFrame.SameSimCode == SameSimSet[NewCodeNum])].index[0]
            resultFrame.loc[rowNumSameSimFound,'Comments'] = "Present Both in MD and Autoproposal"
        except:
            try:
                rownum = rownum+1
                resultFrame.loc[rownum].NewCode = NewCodeSet[NewCodeNum]
                resultFrame.loc[rownum].NewCodeDesc = NewCodeDescSet[NewCodeNum]
                resultFrame.loc[rownum].SameSimCode = SameSimSet[NewCodeNum]
                resultFrame.loc[rownum].CPTDesc = SimCodeDescSet[NewCodeNum]
                resultFrame.loc[rownum].Comments = "Present only by MD"
            except:
                "Nothing"
    except:
        print(NewCodeSet[NewCodeNum])
        #print(i)
        test_set2 = word_tokenize(i)
        #tokenizer = TfidfVectorizer(stop_words = 'english')# initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
        tokenizer = TfidfVectorizer()
        score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
        #score
        pd.set_option("display.max_rows", 11)
        sortedscore = score.sort_values(by=['score'], ascending=False)
        sortedscore.head(11)
        ranknum = 0
        found = "false"
        for x in range(0,10):
            ranknum = ranknum+1
            df = pd.read_csv(MasterSheet) 
            svalue = sortedscore.iloc[x,1]
            GetCPTValue = df.iloc[svalue,0]
            if (GetCPTValue == SameSimSet[NewCodeNum]):
                found = "true"
                break 

        RecNum =0
        for x in range(0,10):
            rownum = rownum+1
            svalue = sortedscore.iloc[x,1]
            Accuracy = sortedscore.iloc[x,2]
            
            Comments = "Present in Autoproposal"

            if (RecNum ==0):
                result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,ranknum,Comments,rownum)
            else:
                result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,"",Comments,rownum)
            RecNum =RecNum +1
           
        try:         
            rowNumSameSimFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum]) & (resultFrame.SameSimCode == SameSimSet[NewCodeNum])].index[0]
            resultFrame.loc[rowNumSameSimFound,'Comments'] = "Present Both in MD and Autoproposal"
        except:
            try:
                rownum = rownum+1 
                resultFrame.loc[rownum].NewCode = NewCodeSet[NewCodeNum]
                resultFrame.loc[rownum].NewCodeDesc = NewCodeDescSet[NewCodeNum]
                resultFrame.loc[rownum].SameSimCode = SameSimSet[NewCodeNum]
                resultFrame.loc[rownum].CPTDesc = SimCodeDescSet[NewCodeNum]
                resultFrame.loc[rownum].Comments = "Present only by MD"
            except:
                "Nothing"

#export_csv = resultFrame.to_csv ('export_dataframe_V4_2018.csv', index = None, header=True)

resultFrame



0045U


NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

SameSimDf = pd.read_csv("Same-Sim_SAMPLE.csv",encoding ='latin1')
sNewCodeDesc = SameSimDf['Sim_LongDesc'].tolist()

# combine documents
#doc_complete = [doc1, doc2, doc3, doc4, doc5]


In [11]:
X =vectorizer.fit_transform(sNewCodeDesc)

In [14]:

from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=3,n_iter=100)
lsa.fit(X)

terms = vectorizer.get_feature_names()


for i,comp in enumerate(lsa.components_):
    termsInComp = zip(terms,comp)
    sortedterms = sorted(termsInComp, key=lambda x: x[1],reverse=True)[:20]
    print("Concept %d:" % i)
    for term in sortedterms:
        print(term[0])
    print(" ")

    

Concept 0:
analysis
or
sequence
variants
performed
and
eg
gene
copy
dna
for
genomic
interrogation
neoplasm
organ
rearrangements
solid
targeted
number
npm1
 
Concept 1:
algorithm
embedded
expression
fixed
formalin
mrna
oncology
paraffin
pcr
profiling
real
rt
score
time
tissue
as
reported
of
utilizing
by
 
Concept 2:
ms
drug
lc
of
gc
date
per
service
any
chromatography
or
all
an
comparison
compounds
drugs
estimated
evaluation
fluid
including
 


In [23]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

import re
import numpy as np
import pandas as pd
from itertools import chain
import string
          
class NormalizeText:
    
     def __init__(self):
          self.trans_table = self.get_no_punctuation_table()
          self.PorterStemmer = PorterStemmer()

     def get_no_punctuation_table(self,translate_to=None):
          not_letters_or_digits = unicode(string.punctuation)+'1234567890'
          translate_table = dict((ord(char), translate_to) for char in not_letters_or_digits)
          return translate_table    
     

     def normalize(self,text):
          orig_text = text
          text = unicode(text)
          trans_table = self.trans_table
          text = text.translate(trans_table)
          text = text.lower()
          text = nltk.word_tokenize(text)
          text = [self.PorterStemmer.stem(word) for word in text if not word in stopwords.words('english')]
          text = " ".join(text)
          return text
     

class MyCorpus:
     def __init__(self, filepath, delim=','):
          self.filepath = filepath
          self.delim = delim
     
     def __iter__(self): #each line assumed to be a separate document
          for line in open(self.filepath):
               yield line.split(self.delim)[1]


class LSA:
     def __init__(self):
          pass

     #singular value decomposition on the term-document matrix
     def fit(self,corpus,min_term_freq,max_df_prop,n_components):
          self.transformer = TfidfVectorizer(min_df=int(min_term_freq),max_df=float(max_df_prop))
          tfidf = self.transformer.fit_transform(corpus) 
          svd = TruncatedSVD(n_components = n_components)
          lsa = svd.fit_transform(tfidf.T)
          self.model = lsa

     
     
class Similarity:
     def __init__(self,transformer,model):
          self.transformer = transformer
          self.model = model
          self.norm = NormalizeText()
     
     #cosine similarity
     def cos_sim(self,X_col,Y_col):
          dot_prod = np.dot(X_col.T,Y_col)
          X_norm = np.sqrt(np.dot(X_col.T,X_col))
          Y_norm = np.sqrt(np.dot(Y_col.T,Y_col))
          cos = dot_prod/(X_norm*Y_norm)
          return cos
     
     def k_closest_terms(self,k,term):
          term = self.norm.normalize(term)
          index = self.transformer.vocabulary_[term]
          
          terms = self.transformer.get_feature_names()
          closestTerms = pd.Series(index=terms)
          
          for i in xrange(len(self.model)):
               closestTerms.loc[terms[i]] = self.cos_sim(self.model[index,:].T,self.model[i,:].T)
               
          closestTerms.sort_values(ascending=False,inplace=True)
                      
          return closestTerms.index[0:k].tolist()          
          
     
     
if __name__=="__main__":
     corpus_file = "Same-Sim_SAMPLE.csv"
min_term_freq = 10 #minimum number of times the word must appear in the corpus
     max_df_prop = 0.10 #maximum proportion of documents in which the word appears
     n_components = 200 #number of components for dimensionality reduction
     
     
     #initialize corpus
     corpus = MyCorpus(corpus_file) #each line in corpus file assumed to be separate document
     
     #fit LSA model
     lsa = LSA()
     lsa.fit(corpus, min_term_freq, max_df_prop,n_components)
     
     #find k most similar terms
     term = 'happiness'
     k = 50
     similarity = Similarity(lsa.transformer,lsa.model)
     k_closest_terms = similarity.k_closest_terms(k=k,term=term)
     print (k_closest_terms)

ValueError: max_df corresponds to < documents than min_df